In [64]:
from Bio import SeqIO
import urllib.parse as up
import os
import psycopg2
import re
from Bio import Entrez
from collections import Counter

Ficheiro GB   # fazer dowload diretamente do site

In [65]:
id_genbank_input = input("ID NCBI")

In [66]:
handle = Entrez.efetch(db='nucleotide',retmode='txt',id=id_genbank_input, rettype = "gb") # dowload do ficheiro
record = SeqIO.read(handle, "gb")

In [67]:
# file = "/home/sa_bruno/Documentos/GitHub/Projeto_LB/ARL15/ARL15.gb"                
# # file = "Teste.gb"
# record = SeqIO.read(file, "genbank") 

Tirar todas as variáveis 

In [68]:
# Genbank
ID = record.name
Organism = record.annotations["organism"]
Accession = record.annotations["accessions"][0]
Definition = record.description
Keywords = ""

for c in record.annotations["keywords"]:
    Keywords += c + ", "
Keywords = Keywords[0: (len(Keywords) -2)]

Sequence = str(record.seq)

In [69]:
# apagar dps   # parte de algoritmos

Count_A = 0
Count_C = 3
Count_T = 3
Count_G = 4

In [70]:
print(ID)
print(Organism)
print(Accession)
print(Definition)
print(Keywords)
print(Sequence)

Z11591
Phytophthora parasitica
Z11591
P.parasitica (Dastur 1896) HIS3 gene for imidazoleglycerol-phosphate dehydratase
HIS3 gene, imidazoleglycerolphosphate dehydratase
GAATTCGAGCTCGGTACCCGGGATCTTGGGCAAATCCAAAGCCGAAGTCACATATACTTGCCGGAAACCTACTCTTCATCATAGCACCCAACCACACATACTTCCTGCCTTTCAGCAGCCCCAACGCTTTGGTTCTGGGGGCTTGGACGTCTCGGTAATCGCTCGTAAATTCCCGCTCGATCAGCCTCGCCACTTCGCCAACGAGGCATCTTTTGGCAAACTCAACCGGAGTGTTTTCACGTGCGGATATCGAACCACCTGACAGCAAATTGTATACGTCGAATCTTGGTGGCGTAGATCCATTTCGGAAATGAAGCAGAATAAATCTTAAGTCCTCATTAAAAATAATGTTGGCCCAAACTGTGGTACACATAATTTATGAATACGTTTGCAATTCTGATTACTTGCTGTCTATATTTGCATTTATGACATGTCTGTTTGGCCAACCATGTGTACCAATACATAGGTACAGTTTCCGTCAAACACGCTTTATCCATTCTGCCAAAGACCACCACAACGATGGCGTCTCCCGTGCAGGCTTTGCTGCTGGACATGGACGGCGTGATGGCCGAGGTGTCCCAGTCCTACCGCCAAGCCATCATCGACACTGCGCGCCACTTCGGCGTGTCCGTCACGCACGAGGACATCGATCACACAAAACTAGCAGGCAATGCCAACAATGACTGGCAGTTGACGCACCGTCTGGTCGTCGACGGGCTCAACGGCGCCAGCTCTGCCCCGACGCTAGAAGCCGTCACGGCTCAATTCGAGGCTCTGTATCAAGGCGTGGGCAACACACTAGGTCTGTGTGAGCTCGAGACGCTGCTTACG

In [71]:
# Locus 
Size = len(record.seq)
Molecular_type = record.annotations["molecule_type"] + " " + record.annotations["topology"]
Genbank_division = record.annotations["data_file_division"] 
Modification_date = record.annotations["date"]

In [72]:
print(Size)
print(Molecular_type)
print(Genbank_division)
print(Modification_date)

1936
DNA linear
PLN
22-JAN-1992


In [73]:
# Sequences

In [74]:
def get_authors(a):
    exis_5 = re.search(r"authors:\s+([\w\s,\.-]+)", a)          #dúvida (como é que o ciclo sabe se isto está vazio ou não)       
    if exis_5:
        Authors = []
        Authors_work = []
        y = re.search(r"authors:\s+([\w\s,\.-]+)", a)    # isto aqui é o remark
        Authors_work.append(y.group(1))
        # else:
        #     Consortium.append("------")
        help_authors = [] 
        Authors = str(Authors_work[0]).split("., ")
        for i, c in enumerate(Authors):
            f = c.split(" and ")
            help_authors.append([i, f])
        c = len(Authors)
        i = 0
        while c > 0:
            if len(help_authors[i][1]) > 1:
                Authors.remove(Authors[i])
                for o in help_authors[i][1]:
                    Authors.append(o)

            c -= 1
            i += 1
    else:
        Authors = []

    return Authors



In [75]:
# referencias  # faz parte do PubMed INfo
Pubmed = []
Title = []
Journal = []
Consortium = []
Remark = []
Authors = []
b = ""
pubmed_trash = record.annotations["references"]    # resolver isto 
c = len(pubmed_trash)
i = 0
while c > 0:
    a = str(pubmed_trash[i]).replace("\n", "§")   
    
    exist_3 = re.search(r"pubmed id:\s+(\d+)", a)
    if exist_3:
        m = re.search(r"pubmed id:\s+(\d+)", a)                 # COnfirmar PubMEd haver sempre/nao
        Pubmed.append(m.group(1))
    else:
        Pubmed.append("------")

    
    n = re.search(r"title:\s+([\w\s]+)", a)
    Title.append(n.group(1))

    exist_2 = re.search(r"journal:\s+([\w\s.()--,]+)", a)
    if exist_2:
        j = re.search(r"journal:\s+([\w\s.()--,]+)", a)
        Journal.append(j.group(1))
    else:
        Journal.append("------")

    exis = re.search(r"consrtm:\s+([\w\s]+)", a)
    if exis:
        k = re.search(r"consrtm:\s+([\w\s]+)", a)    # isto aqui é o remark
        Consortium.append(k.group(1))
    else:
        Consortium.append("------")         # ter cuidado com isto dps
    

    exis_1 = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\s\d])", a)
    if exis_1:
        l = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\d\s:\[;\(\)\.,!?<>{}\"]+)", a)
        Remark.append(l.group(1))
    else:
        Remark.append("------") 

    Authors.append(get_authors(a))


    
    c -= 1
    i += 1

In [76]:
a = record.annotations["references"][0]  #para usar dps
a.authors


'Baltrusch-Weiter,M., Karlovsky,P. and Prell,H.H.'

In [77]:
print(Authors)

[['Baltrusch-Weiter,M', 'Karlovsky,P.', 'Prell,H.H.'], ['Karlovsky,P.']]


In [78]:
print(Pubmed)
print(Journal)
print(Consortium)
print(Remark)
print(Authors)

['------', '------']
['Unpublished', 'Submitted (17-JAN-1992) Karlovsky P., Institute of Plant Pathology, University of Goettingen, Grisebachstrasse 6, Goettingen, Germany']
['------', '------']
['------', '------']
[['Baltrusch-Weiter,M', 'Karlovsky,P.', 'Prell,H.H.'], ['Karlovsky,P.']]


In [79]:
Abstract = []                        # mudar para cada referenias 
DOI = []
Affiliation = []
Entrez.email = 'sa.bruno.2001@gmail.com'
# handle = Entrez.efetch(db='pubmed',retmode='xml',id="15496913")
handle = Entrez.efetch(db='pubmed',retmode='xml',id="8743090")
resultado = Entrez.read(handle)
q = str(resultado["PubmedArticle"][0])
q

'{\'MedlineCitation\': DictElement({\'OtherID\': [StringElement(\'120881\', attributes={\'Source\': \'PIP\'}), StringElement(\'00262844\', attributes={\'Source\': \'POP\'})], \'KeywordList\': [ListElement([StringElement(\'Africa\', attributes={\'MajorTopicYN\': \'N\'}), StringElement(\'Africa South Of The Sahara\', attributes={\'MajorTopicYN\': \'N\'}), StringElement(\'Arab Countries\', attributes={\'MajorTopicYN\': \'N\'}), StringElement(\'Asia\', attributes={\'MajorTopicYN\': \'N\'}), StringElement(\'Developing Countries\', attributes={\'MajorTopicYN\': \'N\'}), StringElement(\'Diseases\', attributes={\'MajorTopicYN\': \'N\'}), StringElement(\'Eastern Africa\', attributes={\'MajorTopicYN\': \'N\'}), StringElement(\'Ethiopia\', attributes={\'MajorTopicYN\': \'N\'}), StringElement(\'Hiv\', attributes={\'MajorTopicYN\': \'Y\'}), StringElement(\'Hiv Infections\', attributes={\'MajorTopicYN\': \'Y\'}), StringElement(\'India\', attributes={\'MajorTopicYN\': \'N\'}), StringElement(\'Kuwait\

In [80]:
o = re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q)
Abstract.append(o.group(1))

p = re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q)
DOI.append(p.group(1))

t = re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q)
print(t.group(1))



Department of Microbiology, Faculty of Medicine, Kuwait University, Kuwait.


In [81]:
print(Title)
print(Abstract)
print(DOI)

['HIS3 gene of Phytophthora parasitica', 'Direct Submission']
['HIV-1 subtype B isolates have previously been described in India only in the state of Andhra Pradesh, while subtype C isolates have been reported as widespread in the Bombay and Goa regions of India. Gag subtype was determined in HIV-1 isolates from six Indians and one Ethiopian. One Indian was a native of Goa residing in Kuwait, and the others were natives of Bihar, Haryana, West Bengal, and New Delhi states. Five subjects were males aged 20-26 years. The remaining two subjects were females aged 34 and 40. Four of the men acquired HIV through sexual transmission']
['10.1089/aid.1996.12.641']


In [94]:
#sequen

temp_l = []
for c in range(len(record.features)):
    temp_l.append(record.features[c].type)

y = Counter(temp_l)
Gene = y["gene"]
# Regulatory = 
#(ID, CDS, Gene, Regulatory, exons, poli_A-site, misc_feature, mRNA))      # acabar esta parte dps  easy
# so contar apartir daqui 
  # Counter({'gene': 5125, 'CDS': 4997, 'tRNA': 105, 'rRNA': 22, 'source': 1, 'tmRNA': 1})
CDS_count = y["CDS"]
Regulatory = y["regulatory"]
exons = y["exons"]
poli_A_site = y["poli_A-site"]                    #  poli_A-site
misc_feature = y["misc_feature"]
mRNA = y["mRNA"]
print(y)
Location_span = str(record.features[0].location)

Counter({'misc_feature': 2, 'source': 1, 'gene': 1, 'CDS': 1})


In [95]:
print(Gene)
print(CDS_count)
print(Regulatory)
print(poli_A_site)
print(misc_feature)
print(mRNA)
print(Location_span)


1
1
0
0
2
0
[0:1936](+)


In [84]:
# usar para dps
CDS_list = []

ID_cds = []
Translacion_seq = []
Localization = []
ID_protein = []
Protein = []
for c in range(len(record.features)):
    if record.features[c].type == "CDS":
        CDS_list.append(record.features[c])

# Translacion_seq
for c in range(len(CDS_list)):
    ID_cds.append(CDS_list[c])
    Translacion_seq.append(CDS_list[c].qualifiers["translation"])
    Localization.append(CDS_list[c].location)
    ID_protein.append(CDS_list[c].qualifiers["protein_id"])
    try:
        Protein.append(CDS_list[c].qualifiers["product"])
    except:
        Protein.append("------")
    
#(ID_cds, Translacion_seq, Localization, ID_protein, Protein)) 
#    (CDS_list[0]).qualifiers["gene"]

In [85]:
for c in range(len(ID_cds)):
    print(ID_cds[c])
    print()
    print(Translacion_seq[c])
    print()
    print(Localization[c])
    print()
    print(ID_protein[c])
    print()
    print(Protein[c])
    print()
    print()


type: CDS
location: [519:1878](+)
qualifiers:
    Key: EC_number, Value: ['4.2.1.19']
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GOA:P28624', 'InterPro:IPR000807', 'InterPro:IPR005834', 'InterPro:IPR006438', 'InterPro:IPR006543', 'InterPro:IPR020565', 'InterPro:IPR020568', 'UniProtKB/Swiss-Prot:P28624']
    Key: gene, Value: ['HIS3']
    Key: product, Value: ['imidazoleglycerol-phosphate dehydratase']
    Key: protein_id, Value: ['CAA77675.1']
    Key: translation, Value: ['MASPVQALLLDMDGVMAEVSQSYRQAIIDTARHFGVSVTHEDIDHTKLAGNANNDWQLTHRLVVDGLNGASSAPTLEAVTAQFEALYQGVGNTLGLCELETLLTPKGLLRELHRRQPKGMAVVTGRPRKDCAKFLTTHGIEDLFPVQIWLEDCPPKPSPEPILLALKALGVEACHAAMVGDTVDDIIAGRKAGAVAYGVLTPQTYAKSILEQTPAAIGKVLEQVGASVVLTPGLGELLDLVPAVPTASKAPTVNGKSGVREANISRVTKETSISVKLSLDGTGKSKVSSGIGFLDHMLTALAKHSRFDLELDCKGDTWIDDHHTTEDCALTLGEAFDVALGDRAGIARFGSACVPLDEALSRAIVDISSRAHSEINLQLVRPSVGELSSEMITHFFESFASAALXTLHVDVLRGRNDHHRAEASFKALAVALRTAVKHDATAGVPSTKGVLA']


['MASPVQALLLDMDGVMAEVSQSYRQAIIDTARHFGVSVTHEDI

In [90]:
CDSID = 4   # VER ISTO DPS

Ligar Base Dados


In [86]:
import mysql.connector as SQLC   #ligação Bruno

DataBase = SQLC.connect(
   host ="127.0.0.1",
   user ="root",
   password ="login123",
   database = "mydb",
   auth_plugin="mysql_native_password"
)
DataBase.autocommit= True
cur = DataBase.cursor()

#jdbc:mysql://127.0.0.1:3306/?user=root


In [87]:
# import mysql.connector as SQLC   #ligação Luís

# DataBase = SQLC.connect(
#    host ="127.0.0.1",
#    user ="root",
#    password ="*luisbraga123*",
#    database = "mydb",
#    auth_plugin="mysql_native_password"
# )
# cur = DataBase.cursor()



In [110]:

# try:
#     cur.execute("""
#         INSERT INTO Genbank (ID, Organism, Accession, Definition, Keywords)    
#         VALUES ()
#         """,
#         (ID, Organism, Accession, Definition, Keywords))
        
        
# except psycopg2.Error as e:
#     print("Erro na escrita na base de dados: {}".format(e) )    
# finally:
#      cur.close()
ID_fake = 4

In [112]:
try:
        cur.execute(f"""
                INSERT INTO Locus (ID, Size, Molecular_type, Genbank_division, Modification_date)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                    (ID, Size, Molecular_type, Genbank_division, Modification_date))

except:
        print("Erro Povoação Locus")

try:
        cur.execute(f"""
                 INSERT INTO Genbank (ID, Organism, Accession, Definition, Keywords)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                        (ID, Organism, Accession, Definition, Keywords))
except:
        print("Erro Povoação Genbank")


try:

        cur.execute("""
                INSERT INTO Sequences (ID, Sequence, Count_A, Count_C, Count_T, Count_G)    
                VALUES (%s, %s, %s, %s, %s,%s)
                """,
                        (ID, Sequence, Count_A, Count_C, Count_T, Count_G))   

except:
        print("Erro Povoação Sequences")


#     cur.execute("""
#         INSERT INTO Pubmed_Info (ID_pubmed, Title, Abstract, Pubmed_Infocol, Authors)   # saltamos 
#         VALUES (%s, %s, %s, %s, %s)
#         """,
#         (Pubmed, Title, Abstract, DOI, Authors))     # id_pubmed na db
    
#     cur.execute("""
#         INSERT INTO References (ID, Pubmed, Title, Journal, Consortium, Remark)     # saltamos
#         VALUES (%s, %s, %s, %s, %s, %s))
#         """,
#         (ID, Pubmed, Journal, Consortium, Remark))   


        
        

# cur.execute("""
#         INSERT INTO Features (ID, CDS_count, CDSID, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
#         VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#         """,
#                 (ID, CDS_count, CDSID, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span))   






cur.execute("""
        INSERT INTO CDS (CDS_ID, Translacion_seq, Localization, ID_protein, Protein)    
        VALUES (%s, %s, %s, %s, %s)
        """,
        (ID_fake, str(Translacion_seq[0]), str(Localization[0]), str(ID_protein[0]), str(Protein[0]))) 

    
#     cur.execute("""
#         INSERT INTO Authors (ID_authors, Affiliation)    
#         VALUES (%s, %s, %s))
#         """,
#         (Authors, Affiliation))    

Erro Povoação Locus
Erro Povoação Genbank
Erro Povoação Sequences


IntegrityError: 1062 (23000): Duplicate entry '4' for key 'CDS.PRIMARY'